<a href="https://colab.research.google.com/github/lucianaa/ImersaoIA/blob/main/Projeto_ImersaoGemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalação, módulos usados e chave secreta

In [1]:
#instalação manual se necessario
!pip install -U -q google-generativeai
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00


In [4]:
#módulos utillizados
from IPython.display import display
from IPython.display import Markdown
import pathlib
import textwrap
import pandas as pd
import numpy as np
import requests
import PyPDF2
import google.generativeai as genai
from google.colab import userdata

In [5]:
#incluindo key
key = userdata.get('KEY')
genai.configure(api_key=key)

# Escolha do modelo

In [6]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [39]:
model='models/embedding-001'



# Funções utilizadas


In [8]:
#funções úteius
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def embed_fn(title, text):
  model = "models/embedding-001"
  return genai.embed_content(model=model,
                               content=text,
                               title=title,
                               task_type="RETRIEVAL_DOCUMENT")["embedding"]

# Definindo a função para gerar e buscar a consulta
def gerar_e_buscar_consulta(consulta, base, model):
    embedding_da_consulta = genai.embed_content(model=model,
                                               content=consulta,
                                               task_type="RETRIEVAL_QUERY")["embedding"]
    produtos_escalares = np.dot(np.stack(base["Embeddings"]), embedding_da_consulta)
    indice = np.argmax(produtos_escalares)
    return base.iloc[indice]["Conteudo"]

def formatar_texto(trecho, model):
  prompt = f"Reescreva esse texto de corrigindo as palavras que estão separadas, além disso retire números e caracteres especiais fora do contexto, sem adicionar informações que não façam parte do texto: {trecho}"

  response = model.generate_content(prompt)
  return response

# Baixando documento PDF de uma URL

In [55]:
#exemplo: https://documento.ifnmg.edu.br/action.php?kt_path_info=ktcore.actions.document.view&fDocumentId=54935
import io
from urllib.request import Request, urlopen
from PyPDF2 import PdfReader

url = "https://documento.ifnmg.edu.br/action.php?kt_path_info=ktcore.actions.document.view&fDocumentId=54935"
headers = {'User-Agent': 'Mozilla/5.0 (X11; Windows; Windows x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'}

response = requests.get(url=url, headers=headers, timeout=120)
memory_file = io.BytesIO(response.content)
#print(response.content)
pdf_file = PdfReader(memory_file)

# Lendo cada página do PDF e adicionando em um vetor de conteúdo e título para ser adicionado a um dataframe

In [56]:
# Configuração de geração
generation_config = {
    "temperature": 0.5,
    "candidate_count": 1
}

model2 = genai.GenerativeModel("gemini-1.0-pro", generation_config=generation_config)

# Iterar pelas páginas e extrair o texto
all_pages_text = []
titles = []
for page_number in range(len(pdf_file.pages)):
  page = pdf_file.pages[page_number]
  text = page.extract_text ()
  text_response = formatar_texto(text, model2)
  all_pages_text.append(text)
  titles.append("Página " + str(page_number + 1))

#Caso queira mostrar cada página, descomente a linha abaixo
#print(all_pages_text)

# Manipulação do dataframe a partir do documento PDF e criando os embeddings relativo a cada linha/página

In [33]:
df = pd.DataFrame({'Titulo': titles, 'Conteudo': all_pages_text})
# Aplicando a função embed_fn ao DataFrame
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Página 1,Minis tério da E ducação\nSecr etaria de E duc...,"[0.021161817, -0.027249038, -0.025559682, 0.01..."
1,Página 2,\n \n \nCAMPUS ARA CUAÍ\n \nCódigoCursos\nsup...,"[0.02475441, -0.03806406, -0.040256873, -0.001..."
2,Página 3,\nCódigoCursos\nsuperioresAutorização /\nReco...,"[0.013009609, -0.043040063, -0.04180947, 0.016..."
3,Página 4,\nCódigoCursos\nsuperioresAutorização /\nReco...,"[0.00892145, -0.042443924, -0.04696371, 0.0367..."
4,Página 5,3.2.1. Sa  sfazer todas as c ondiç ões do pr e...,"[0.015021474, -0.034581374, -0.05251509, 0.024..."
5,Página 6,5.2. Após a divulg ação da relação preliminar...,"[0.032785352, -0.058065798, -0.058842566, 0.03..."
6,Página 7,8.1.2.1 Para os ﬁns do dispos to no item 8.1.2...,"[0.03065155, -0.051423233, -0.049242962, 0.032..."
7,Página 8,e) Para candida tos que concorrem na modalidad...,"[-0.009143898, -0.046289995, -0.049015082, 0.0..."
8,Página 9,10.3.3. As cópias desses documen tos podem ser...,"[0.006190931, -0.024917925, -0.040336464, 0.02..."
9,Página 10,"14.4.1. Caso necessário , por interesse do IFN...","[0.020378653, -0.05476258, -0.043165788, 0.035..."


# Interagindo com usuário através de chat para saber o quais informações ele deseja a partir do edital

In [34]:
chat = model2.start_chat(history=[])
#response = chat.send_message(prompt)
#to_markdown(response.text)

In [53]:
# Consulta
# Exemplo: "Qual é o período de inscrições?"
prompt = input("Pergunta alguma coisa sobre o edital : ")

trecho = gerar_e_buscar_consulta(prompt, df, model)
# Prompt para reescrita
prompt2 = f"Considere o texto: {trecho} e mostre somentea resposta sucinta da seguinte pergunta: {prompt}"
response = chat.send_message(prompt2)
#print(response)
print("Resposta: \n")

to_markdown(response.text)


Pergunta alguma coisa sobre o edital : Qual é o período de inscrições?
Resposta: 



> 02 de dezembro de 2022 (às 09:00 h) a 05 de fevereiro de 2023 (até as 17:00 h)